# Setup

In [1]:
import os
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate

In [3]:
# Set API key
api_key = os.environ["ANTHROPIC_API_KEY"]

# Initialize models. We'll use a smaller LLM for most of the work, but a large LLM to distill conversations and writing the final report
small_llm = ChatAnthropic(api_key=api_key, model="claude-3-haiku-20240307")
big_llm = ChatAnthropic(api_key=api_key, model="claude-3-5-sonnet-20240620")

# Generating the Outline

## Generate Conversations and Resources

In this section we will fetch related articles and develop a set of personas based on the outlines of the fetched related articles. Once the personas have been created, we will then pass them one by one into a conversation cycle.

In the conversation cycle, the persona will ask questions about the topic to an expert on the topic. The expert will take their questions, split it into search queries, then find resources online based on these queries. The expert will then sift through these, collect the trusted sources, and answer the question using these sources. The persona will read the response and ask another question until they have no more questions. The conversation will be saved for the creation of the final outline.

Once all of the personas have conversed with the expert, we will have compiled a set of conversations and trusted resources to be used in the final outline.

This section will be the bulk of the work.

### Fetch Related Articles

### Create Personas

### Converse With Expert

## Generate Draft Outline

The draft outline is a rough draft of the outline based only on the task. The collected conversations and resources from above will then be used to refine the draft outline.

## Generate Final Outline

Use the collected conversations and resources to refine the draft outline. The result will be the final outline.

# Generate the Final Article

Each section will be generated independently using the previously gathered resources. Once each section is generated, we will have a large model go over the entire article to ensure consistency and remove duplicate information.